# 🔄 3.4 Data Transformation

In this notebook you’ll learn how to **transform** data so it’s easy to analyse and model. We’ll work with `hippo_nutrients.csv` and cover filtering, grouping, reshaping (wide/long), pivoting, and numeric transformations such as **log** and **z-score** standardisation.

## 🎯 Objectives
- Filter rows and select columns with expressive, reproducible code.
- Group and aggregate (mean, median, multiple aggregations).
- Reshape between **wide** and **long** formats (melt / pivot).
- Pivot-tabulate values for quick comparisons.
- Create derived variables and apply **log**, **z-score**, **min–max** scaling.
- (Bonus) Group-wise standardisation and rolling transforms over time.

## 📌 Context
Transformation is the ladder from raw data to insight. In nutrition datasets, you’ll often transform by **group** (e.g., nutrient, sex, year) and reshape between **wide** (one row per ID with many columns) and **long** (one row per measurement).

> **Wide vs Long**
- **Wide**: Each variable has its own column (e.g., `Iron`, `Calcium`, `Vitamin_D` columns). Good for human reading, some stats.
- **Long (tidy)**: One row per observation; variables encoded in *columns*, not headers (e.g., a `Nutrient` column and a `Value` column). Preferred for plotting, grouping, modelling.

<details><summary>Fun Fact</summary>
Transforming data is like a hippo rearranging its snacks — same nutrients, better view! 🦛
</details>

In [ ]:
# Setup for Google Colab: Fetch datasets automatically or manuallyimport osfrom google.colab import filesMODULE = '03_data_handling'DATASET = 'hippo_nutrients.csv'BASE_PATH = '/content/data-analysis-projects'MODULE_PATH = os.path.join(BASE_PATH, 'notebooks', MODULE)DATASET_PATH = os.path.join('data', DATASET)try:    print('Attempting to clone repository...')    if not os.path.exists(BASE_PATH):        !git clone https://github.com/ggkuhnle/data-analysis-projects.git    print('Setting working directory...')    os.chdir(MODULE_PATH)    if os.path.exists(DATASET_PATH):        print(f'Dataset found: {DATASET_PATH} ✅')    else:        raise FileNotFoundError('Dataset missing after clone.')except Exception as e:    print(f'Cloning failed: {e}')    print('Falling back to manual upload...')    os.makedirs('data', exist_ok=True)    uploaded = files.upload()    if DATASET in uploaded:        with open(DATASET_PATH, 'wb') as f:            f.write(uploaded[DATASET])        print(f'Successfully uploaded {DATASET} ✅')    else:        raise FileNotFoundError(f'Upload failed. Please upload {DATASET}.')%pip install -q pandas numpyimport pandas as pd, numpy as nppd.set_option('display.max_columns', 30)print('Python environment ready.')

## 1) Load and Inspect
Let’s load `hippo_nutrients.csv` and inspect the first rows and data types.

In [ ]:
df = pd.read_csv('data/hippo_nutrients.csv')print('Shape:', df.shape)print('Dtypes:\n', df.dtypes)display(df.head(5))

## 2) Filtering Rows and Selecting Columns
Use **boolean masks** to keep the rows you want, and bracket notation to select columns. This is the bread-and-butter of transformation.

In [ ]:
# Example: female hippos, Iron only, years ≥ 2024mask = (df['Sex'] == 'F') & (df['Nutrient'] == 'Iron') & (df['Year'] >= 2024)cols = ['ID', 'Year', 'Nutrient', 'Value', 'Age', 'Sex']df_female_iron = df.loc[mask, cols].sort_values(['ID', 'Year']).reset_index(drop=True)display(df_female_iron.head(5))

## 3) Grouping and Aggregation
Summarise data by groups (e.g., nutrient, sex, or year). You can apply single or multiple aggregations.

### Common patterns
- `df.groupby('Nutrient')['Value'].mean()`
- `df.groupby(['Nutrient','Sex']).agg({'Value': ['mean','median','count']})`

In [ ]:
# Mean Value by Nutrientmean_by_nutrient = df.groupby('Nutrient')['Value'].mean().sort_values(ascending=False)display(mean_by_nutrient)# Multiple aggregations by Nutrient and Sexagg_ns = (    df.groupby(['Nutrient','Sex'])      .agg(Mean_Value=('Value','mean'),           Median_Value=('Value','median'),           N=('Value','size'))      .reset_index())display(agg_ns.head(8))

## 4) Reshaping Between Wide and Long
Many real datasets arrive in **wide** form; most analyses and plots like **long** (tidy) form.

### Long (tidy) → one row per observation
- Columns: `ID`, `Year`, `Age`, `Sex`, `Nutrient`, `Value`

### Wide → columns are variables (e.g., one column per nutrient)
- Columns: `ID`, `Year`, `Age`, `Sex`, `Iron`, `Calcium`, `Vitamin_D`, ...

### 4.1 Long → Wide (pivot wider)
Use `pivot_table` with `index` = identifier columns and `columns` = the variable you want as new columns (`Nutrient`). Values come from `Value`.

- Pros: human-readable, one row per subject-year.
- Cons: harder to group/plot by Nutrient unless you re-melt later.

In [ ]:
id_cols = ['ID','Year','Age','Sex']wide = df.pivot_table(index=id_cols, columns='Nutrient', values='Value', aggfunc='mean')wide = wide.reset_index()  # move index back to columnsdisplay(wide.head(5))

### 4.2 Wide → Long (melt)
Use `melt` to gather nutrient columns back into two columns: `Nutrient` (variable name) and `Value` (measurement). This is the canonical *tidy* structure for plotting, modelling, and faceting.

In [ ]:
value_vars = [c for c in wide.columns if c not in id_cols]  # all nutrient columnslong_again = wide.melt(id_vars=id_cols, value_vars=value_vars,                      var_name='Nutrient', value_name='Value')display(long_again.head(6))

## 5) Pivot Tables for Comparisons
Quickly cross-tabulate means by two dimensions (e.g., `Nutrient` × `Year`).

In [ ]:
pt = df.pivot_table(values='Value', index='Nutrient', columns='Year', aggfunc='mean')display(pt)

## 6) Derived Variables and Numeric Transformations
Real analyses often need **transformed** variables to meet model assumptions or to compare across scales.

### 6.1 Log transform
Use when data are **right-skewed** (e.g., highly variable intakes). Add a small `ε` to avoid `log(0)`.

### 6.2 z-score standardisation
Convert values to standard units: `(x - mean) / std`. Helpful when combining variables on different scales.

### 6.3 Min–max scaling (0–1)
Maps the minimum to 0 and maximum to 1. Useful for comparability in dashboards.

### 6.4 Group-wise transforms
Standardise **within** groups (e.g., within each `Nutrient`) so each nutrient’s distribution is centred and scaled separately.

In [ ]:
df_trans = df.copy()# 6.1 Log transform: add epsilon to avoid log(0) if presenteps = 1e-6df_trans['Value_log'] = np.log(df_trans['Value'] + eps)# 6.2 Global z-scoremu = df_trans['Value'].mean()sd = df_trans['Value'].std(ddof=0)df_trans['Value_z'] = (df_trans['Value'] - mu) / (sd if sd else 1.0)# 6.3 Min–max scalingvmin, vmax = df_trans['Value'].min(), df_trans['Value'].max()rng = vmax - vmin if vmax > vmin else 1.0df_trans['Value_minmax'] = (df_trans['Value'] - vmin) / rng# 6.4 Group-wise z-score by Nutrientdf_trans['Value_z_by_nutrient'] = (    df_trans.groupby('Nutrient')['Value']           .transform(lambda s: (s - s.mean()) / (s.std(ddof=0) if s.std(ddof=0) else 1.0)))display(df_trans.head(8))

## 7) (Bonus) Time-based and Rolling Transforms
For longitudinal data (e.g., the same `ID` measured across `Year`), we can compute rolling means or deltas. This is illustrative if your dataset has multiple years per ID.

In [ ]:
df_roll = (    df.sort_values(['ID','Nutrient','Year'])      .groupby(['ID','Nutrient'], as_index=False)      .apply(lambda g: g.assign(          Value_roll_mean=g['Value'].rolling(window=2, min_periods=1).mean(),          Value_delta=g['Value'].diff()      )))display(df_roll.head(10))

## 🧪 Exercise 1 — Grouped Summary
Filter to `Nutrient == 'Vitamin_D'`, then compute **median** `Value` by `Sex` and `Year`. Present the result sorted by `Year` and then `Sex`.

**Hints**:
- Filter: `df[df['Nutrient'] == 'Vitamin_D']`
- Group and aggregate: `.groupby(['Sex','Year'])['Value'].median().reset_index()`
- Sort: `.sort_values(['Year','Sex'])`

In [ ]:
# Your code here

## 🧪 Exercise 2 — Wide ↔ Long
1) Create a **wide** table with index `[ID, Year]` and columns as nutrients (values = mean `Value`).  
2) Convert that wide table **back to long** using `melt` with `Nutrient` and `Value` columns.

**Hints**:
- `pivot_table(index=['ID','Year'], columns='Nutrient', values='Value', aggfunc='mean')`
- `melt(id_vars=['ID','Year'], var_name='Nutrient', value_name='Value')`

In [ ]:
# Your code here

## 🧪 Exercise 3 — Transformations for Modelling
Create a dataframe with these new columns on the **Iron** subset only:
- `Value_log_iron` — log-transformed value with epsilon.
- `Value_z_iron_by_sex` — z-score **within Sex**.
- `Value_minmax_iron_by_year` — min–max scaling **within Year**.

**Hints**:
- Filter: `(df['Nutrient'] == 'Iron')`
- Group-wise transforms with `.groupby(...).transform(...)`

In [ ]:
# Your code here

## ✅ Conclusion & Next Steps
You’ve practised the core transformation tools:
- Filtering and selecting for clear subsets.
- Grouping and aggregating for summaries.
- Reshaping between **wide** and **long** for tidy analysis.
- Pivot tables for quick comparisons.
- Numeric transformations: **log**, **z-score**, **min–max**, and **group-wise** standardisation.

👉 Next: **3.5 Data Aggregation** — robust summaries, grouped statistics, and combining pipelines for analysis-ready datasets.

**Resources**:
- Pandas GroupBy: https://pandas.pydata.org/docs/user_guide/groupby.html
- Reshaping (melt/pivot): https://pandas.pydata.org/docs/user_guide/reshaping.html
- Working with dtypes: https://pandas.pydata.org/docs/user_guide/basics.html#basics-dtypes